In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import *
from pyspark.sql.types import *
import pyspark.sql.functions as F
import time

In [0]:
### Variablen laden

### Webpages fuer Datensatz Motorisierter Individualverkehr von 2012 - 2020 (https://data.stadt-zuerich.ch/dataset/sid_dav_verkehrszaehlung_miv_od2031)

url_12 = "https://data.stadt-zuerich.ch/dataset/6212fd20-e816-4828-a67f-90f057f25ddb/resource/a0af24df-7b7f-4951-a08b-9391778c92d7/download/sid_dav_verkehrszaehlung_miv_od2031_2012.csv"

url_13 = "https://data.stadt-zuerich.ch/dataset/6212fd20-e816-4828-a67f-90f057f25ddb/resource/4b5aa585-a9f6-4cda-b626-4878d7c954bf/download/sid_dav_verkehrszaehlung_miv_od2031_2013.csv"

url_14 = "https://data.stadt-zuerich.ch/dataset/6212fd20-e816-4828-a67f-90f057f25ddb/resource/cdb43ead-a065-40a5-9565-56f6cfe95b5e/download/sid_dav_verkehrszaehlung_miv_od2031_2014.csv"

url_15 = "https://data.stadt-zuerich.ch/dataset/6212fd20-e816-4828-a67f-90f057f25ddb/resource/62457f7f-c247-4b87-beee-5677d98d7cf4/download/sid_dav_verkehrszaehlung_miv_od2031_2015.csv"

url_16 = "https://data.stadt-zuerich.ch/dataset/6212fd20-e816-4828-a67f-90f057f25ddb/resource/f454b4d5-7570-4547-8ee0-2bde785ec628/download/sid_dav_verkehrszaehlung_miv_od2031_2016.csv"

url_17 = "https://data.stadt-zuerich.ch/dataset/6212fd20-e816-4828-a67f-90f057f25ddb/resource/f873cc29-96ac-4b2f-b175-f733513e4012/download/sid_dav_verkehrszaehlung_miv_od2031_2017.csv"

url_18 = "https://data.stadt-zuerich.ch/dataset/6212fd20-e816-4828-a67f-90f057f25ddb/resource/d5963dee-7841-4e64-9268-6c850a2fc497/download/sid_dav_verkehrszaehlung_miv_od2031_2018.csv"

url_19 = "https://data.stadt-zuerich.ch/dataset/6212fd20-e816-4828-a67f-90f057f25ddb/resource/fa64fa70-6328-4d47-bcf0-1eff694d7c22/download/sid_dav_verkehrszaehlung_miv_od2031_2019.csv"

url_20 = "https://data.stadt-zuerich.ch/dataset/6212fd20-e816-4828-a67f-90f057f25ddb/resource/44607195-a2ad-4f9b-b6f1-d26c003d85a2/download/sid_dav_verkehrszaehlung_miv_od2031_2020.csv"

In [0]:
### Daten via Panda laden 
miv_2012 = pd.read_csv(url_12, sep=",", header=0, index_col=0)
miv_2013 = pd.read_csv(url_13, sep=",", header=0, index_col=0)
miv_2014 = pd.read_csv(url_14, sep=",", header=0, index_col=0)
miv_2015 = pd.read_csv(url_15, sep=",", header=0, index_col=0)
miv_2016 = pd.read_csv(url_16, sep=",", header=0, index_col=0)
miv_2017 = pd.read_csv(url_17, sep=",", header=0, index_col=0)
miv_2018 = pd.read_csv(url_18, sep=",", header=0, index_col=0)
miv_2019 = pd.read_csv(url_19, sep=",", header=0, index_col=0)
miv_2020 = pd.read_csv(url_20, sep=",", header=0, index_col=0)

/databricks/python/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,14) have mixed types.Specify dtype option on import or set low_memory=False.
 interactivity=interactivity, compiler=compiler, result=result)

In [0]:
### Alle Dataframes zusammenfügen von 2012 bis 2020
miv_pa_total = pd.concat([miv_2012, miv_2013, miv_2014, miv_2015, miv_2016, miv_2017, miv_2018, miv_2019, miv_2020], axis=0)


In [0]:
### Nur für Pandas Dataframe rechnen
miv_pa_total = miv_2012

In [0]:
### Datenframe anzeigen
len(miv_pa_total)
miv_pa_total.head(5)
type(miv_pa_total)

Out[76]: pandas.core.frame.DataFrame

In [0]:
### Umrechnung Koordinaten mit Pandas Dataframe
miv_pa_total_pd = miv_pa_total
type(miv_pa_total_pd)
miv_pa_total_pd.head(3)

Out[95]:

,MSName,ZSID,ZSName,Achse,HNr,Hoehe,EKoord,NKoord,Richtung,Knummer,Kname,AnzDetektoren,D1ID,D2ID,D3ID,D4ID,MessungDatZeit,LieferDat,AnzFahrzeuge,AnzFahrzeugeStatus
MSID,,,,,,,,,,,,,,,,,,,,
Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2012-01-01T00:00:00,2020-02-03,NaN,Fehlend
Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2012-01-01T01:00:00,2020-02-03,256.0,Gemessen
Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2012-01-01T02:00:00,2020-02-03,186.0,Gemessen


In [0]:
### Nur die relevanten Spalten auswählen
miv_pa_total = miv_pa_total[['EKoord', 'NKoord']]
miv_pa_total = miv_pa_total.to_csv()

In [0]:
### Das aggregierte File als CSV abspeichern
dbutils.fs.rm("/FileStore/tables/g10/miv_pa_total.csv", True)  
dbutils.fs.put("/FileStore/tables/g10/miv_pa_total.csv", miv_pa_total)

Wrote 413654323 bytes.
Out[78]: True

In [0]:
#Funktionen initialisieren für die Umrechnung der Koordinaten
#
# Convert CH y/x to WGS lat
def CHtoWGSlat(y, x):
    # Axiliary values (% Bern)
    y_aux = (y - 600000) / 1000000
    x_aux = (x - 200000) / 1000000
    lat = (16.9023892 + (3.238272 * x_aux)) + \
            - (0.270978 * pow(y_aux, 2)) + \
            - (0.002528 * pow(x_aux, 2)) + \
            - (0.0447 * pow(y_aux, 2) * x_aux) + \
            - (0.0140 * pow(x_aux, 3))
    # Unit 10000" to 1" and convert seconds to degrees (dec)
    lat = (lat * 100) / 36
    return lat

# Convert CH y/x to WGS long
def CHtoWGSlng(y, x):
    # Axiliary values (% Bern)
    y_aux = (y - 600000) / 1000000
    x_aux = (x - 200000) / 1000000
    lng = (2.6779094 + (4.728982 * y_aux) + \
            + (0.791484 * y_aux * x_aux) + \
            + (0.1306 * y_aux * pow(x_aux, 2))) + \
            - (0.0436 * pow(y_aux, 3))
    # Unit 10000" to 1" and convert seconds to degrees (dec)
    lng = (lng * 100) / 36
    return lng
  

In [0]:
# Convert to a UDF Function by passing in the function and return type of function
udfCHtoWGSlat = F.udf(CHtoWGSlat, FloatType())

# Convert to a UDF Function by passing in the function and return type of function
udfCHtoWGSlng = F.udf(CHtoWGSlng, FloatType())

In [0]:
### Pandas Dataframe nur mit relevanten Spalten vorbereiten
miv_pa_total_pd = pd.DataFrame(miv_pa_total_pd[['EKoord','NKoord']])

### Umrechnung und Performance-Messung mit Dataframe Pandas
### Measuring the time
start = time.time()

### Convert coordinates to the international format
lon = []
for i in range(0,len(miv_pa_total_pd['EKoord'])):
    lon.append(CHtoWGSlng(miv_pa_total_pd['EKoord'][i], miv_pa_total_pd['NKoord'][i]))
    i += 1

lat = []
for i in range(0,len(miv_pa_total_pd['NKoord'])):
    lat.append(CHtoWGSlat(miv_pa_total_pd['EKoord'][i], miv_pa_total_pd['NKoord'][i]))
    i += 1

miv_pa_total_pd.insert(0,'latitude',lat)
miv_pa_total_pd.insert(0,'longitude',lon)

### Measuring the time
stop = time.time()
### Show the time it takes
print ("Time: ", (stop-start), " seconds.")
um_koor_pd = (stop-start)

Time: 189.12059903144836 seconds.

In [0]:
### Pandas Dataframe anzeigen
miv_pa_total_pd.head(5)

Out[99]:

,longitude,latitude,EKoord,NKoord
MSID,,,,
Z001M001,39.310955,52.461194,2683009.89,1243936.2
Z001M001,39.310955,52.461194,2683009.89,1243936.2
Z001M001,39.310955,52.461194,2683009.89,1243936.2
Z001M001,39.310955,52.461194,2683009.89,1243936.2
Z001M001,39.310955,52.461194,2683009.89,1243936.2


In [0]:
### CSV File vom HFDS laden und im Cache speichern
koordinatenDF = spark.read.options(header='true').option('inferSchema', 'true').csv("/FileStore/tables/g10/miv_pa_total.csv")
koordinatenDF.cache()

Out[81]: DataFrame[MSID: string, EKoord: double, NKoord: double]

In [0]:
### Dataframe anzeigen
koordinatenDF.take(3)
koordinatenDF.printSchema()

root
-- MSID: string (nullable = true)
-- EKoord: double (nullable = true)
-- NKoord: double (nullable = true)

In [0]:
### Remove Parquet-File if it exists
dbutils.fs.rm("/FileStore/koordinaten.parquet", True)

### DataFrames can be saved as Parquet files, maintaining the schema information.
koordinatenDF.write.parquet("/FileStore/koordinaten.parquet")

In [0]:
### Read in the Parquet file created above.
koordinatenPQ = spark.read.parquet("/FileStore/koordinaten.parquet")

In [0]:
### Umrechnung und Performance-Messung mit Parquet-Format
### Measuring the time
start = time.time()

### Convert coordinates to the international format
koordinaten_lon = koordinatenPQ.withColumn("longitude", udfCHtoWGSlng(F.col("EKoord"), F.col("NKoord")))

koordinaten_lon_and_lat = koordinaten_lon.withColumn("latitude", udfCHtoWGSlat(F.col("EKoord"), F.col("NKoord")))

### Measuring the time
stop = time.time()
### Show the time it takes
print ("Time: ", (stop-start), " seconds.")
um_koor_pq = (stop-start)

Time: 0.09922242164611816 seconds.

In [0]:
### Parquet-Dataframe anzeigen mit umgerechnete Koordinaten
koordinaten_lon_and_lat.take(3)

Out[86]: [Row(MSID='Z090M002', EKoord=2686453.0, NKoord=1251269.0, longitude=39.405277252197266, latitude=52.50947570800781),
 Row(MSID='Z090M002', EKoord=2686453.0, NKoord=1251269.0, longitude=39.405277252197266, latitude=52.50947570800781),
 Row(MSID='Z090M002', EKoord=2686453.0, NKoord=1251269.0, longitude=39.405277252197266, latitude=52.50947570800781)]

In [0]:
### Generiere Spark Context
sc = spark.sparkContext

In [0]:
### Lade CSV File mit Verteilung auf 4 Nodes
csvFile = sc.textFile("/FileStore/miv_pa_total.csv", 4)
type(csvFile)

Out[88]: pyspark.rdd.RDD

In [0]:
### Header muss entfernt werden, damit die ganze Spalte berechnet werden kann (ohne Strings)
csvFileheader = csvFile.first()
header = sc.parallelize([csvFileheader])
csvFile = csvFile.subtract(header)

In [0]:
### CSV File spliten auf einzelne Spalten
parts = csvFile.map(lambda l: l.split(","))
parts.take(5)

Out[90]: [['Z023M003', '2678391.41', '1250163.66'],
 ['Z023M003', '2678391.41', '1250163.66'],
 ['Z023M003', '2678391.41', '1250163.66'],
 ['Z023M003', '2678391.41', '1250163.66'],
 ['Z023M003', '2678391.41', '1250163.66']]

In [0]:
### Einzelne Zeilen in entsprechendes DataType umwandeln
koordinaten = parts.map(lambda p: Row(koordinate_ost=float(p[1]),koordinate_nord=float(p[2])))
koordinaten.take(5)

Out[91]: [Row(koordinate_ost=2678391.41, koordinate_nord=1250163.66),
 Row(koordinate_ost=2678391.41, koordinate_nord=1250163.66),
 Row(koordinate_ost=2678391.41, koordinate_nord=1250163.66),
 Row(koordinate_ost=2678391.41, koordinate_nord=1250163.66),
 Row(koordinate_ost=2678391.41, koordinate_nord=1250163.66)]

In [0]:
### Dataframe generieren
csvFiledf = spark.createDataFrame(koordinaten)
csvFiledf.take(3)

Out[92]: [Row(koordinate_ost=2678391.41, koordinate_nord=1250163.66),
 Row(koordinate_ost=2678391.41, koordinate_nord=1250163.66),
 Row(koordinate_ost=2678391.41, koordinate_nord=1250163.66)]

In [0]:
### Erste Umrechnung und Performance-Messung mit Dataframe
### Measuring the time
start = time.time()

### Convert coordinates to the international format
csvFiledf_lon = csvFiledf.withColumn("longitude", udfCHtoWGSlng(F.col("koordinate_ost"), F.col("koordinate_nord")))

csvFiledf_lon_and_lat = csvFiledf_lon.withColumn("latitude", udfCHtoWGSlat(F.col("koordinate_ost"), F.col("koordinate_nord")))

### Measuring the time
stop = time.time()
### Show the time it takes
print ("Time: ", (stop-start), " seconds.")
um_koor_df = (stop-start)

Time: 0.033425092697143555 seconds.

In [0]:
### Zeit für Pandas Dataframe extrapolieren (x9)
um_koor_pd_ex = um_koor_pd * 9
### Gemessene Zeiten in Dataframe speichern
schema = StructType([StructField("Format", StringType(), True),
                     StructField("Umrechnung in s", FloatType(), True)])

dataset = [("Pandas", round(um_koor_pd_ex, 2)), 
           ("Spark DF", round(um_koor_df, 2)), 
           ("Parquet", round(um_koor_pq, 2))]
            
zeiten_df = spark.createDataFrame(dataset, schema)
zeiten_df.show()

+--------+---------------+
 Format|Umrechnung in s|
+--------+---------------+
 Pandas| 1702.09|
Spark DF| 0.03|
 Parquet| 0.1|
+--------+---------------+

In [0]:
### Ploten der Zeiten
zeiten_df.display()

Format,Umrechnung in s
Pandas,1702.09
Spark DF,0.03
Parquet,0.1
